## RQ2 - Coleção dos autores com respetivos commits e datas a um dada linha

In [ ]:
import subprocess
from subprocess import Popen
import pandas as pd
import numpy as np
import glob
import sys
from pathlib import Path
import ast
import astpretty
import astor


In [ ]:
buggy_elements = pd.read_csv('../data/RQ1_Data_Processed.csv', sep=',')

In [ ]:
buggy_elements.head()

In [ ]:
from datetime import datetime

buggy_elements_df = pd.DataFrame()

length = buggy_elements.shape[0]


for i in range(10000):
    sha1 = (buggy_elements['Hash'].iloc[i])
    bug_id = (buggy_elements['Bug ID'].iloc[i])
    bug_type = (buggy_elements['Bug Type'].iloc[i])
    bug_repo = (buggy_elements['Repo'].iloc[i])
    bug_file = (buggy_elements['File'].iloc[i])
    bug_line = (buggy_elements['Line'].iloc[i])
    bug_component = (buggy_elements['Component'].iloc[i])
    split_repo = (buggy_elements['File'].iloc[i]).rsplit('/', 1)
    #print(split_repo[0])
    #print(split_repo[1])
    #print(bug_repo)


### Fazer checkout para a versão pretendida
    try:
        process = subprocess.Popen(['git','checkout',sha1],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + bug_repo, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
        output, errors = process.communicate()


    ### Fazer git log para a linha 

        process2 = subprocess.Popen(['git','log','-L', str(bug_line) + ',' + str(bug_line)+ ':' +str(split_repo[1])],cwd = r'C:\Users\migue\OneDrive\Ambiente de Trabalho\teste' + '\\' + str(bug_repo) + '\\' + str(split_repo[0]), stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True, shell = True)
        output2, errors = process2.communicate()
        #print(output2.splitlines())

        commits = []
        authors = []
        dates = []

        for linha in output2.splitlines():



            if linha.startswith('commit'):

                commit = linha.split("commit")[1].strip()
                print(commit)
                commits.append(commit)

            if linha.startswith('Author:'):

                author = linha.split("Author:")[1].strip().split('<',1)[0]
                print(author)
                authors.append(author)

            if linha.startswith('Date:'):

                date = linha.split("Date:")[1].strip()
                parsed_date = date.split()
                print(parsed_date)
                day = parsed_date[2]
                year = parsed_date[4]

                if ( parsed_date[1] == 'Jan'):
                    month = 1
                if ( parsed_date[1] == 'Feb'):
                    month = 2
                if ( parsed_date[1] == 'Mar'):
                    month = 3
                if ( parsed_date[1] == 'Apr'):
                    month = 4
                if ( parsed_date[1] == 'May'):
                    month = 5
                if ( parsed_date[1] == 'Jun'):
                    month = 6
                if ( parsed_date[1] == 'Jul'):
                    month = 7
                if ( parsed_date[1] == 'Aug'):
                    month = 8
                if ( parsed_date[1] == 'Set'):
                    month = 9
                if ( parsed_date[1] == 'Oct'):
                    month = 10
                if ( parsed_date[1] == 'Nov'):
                    month = 11
                if ( parsed_date[1] == 'Dec'):
                    month = 12


                datetime_format = day + '/' + str(month) + '/' + year + ' ' + parsed_date[3]

                print(datetime_format)

                datetime = datetime.strptime(datetime_format, '%d/%m/%Y %H:%M:%S')


                dates.append(datetime)

        row  = {'Bug ID': bug_id,'Hash': sha1,'Bug Type': bug_type ,'Repo' : bug_repo ,'File': bug_file, 'Line': bug_line, 'Components': bug_component, 'Commits' : commits, 'Authors' : authors, 'Dates' : dates}
        buggy_elements_df = buggy_elements_df.append(row, ignore_index = True)
        
        print ('i')
        print(i)
        
        
    except:
        
        print(' An error occured')

        





In [ ]:
buggy_elements_df.shape

In [ ]:
buggy_elements_df.head()

## Data processing

In [ ]:
buggy_elements_df['Lifetime']= " "

In [ ]:
buggy_elements_df['Number of Authors']= " "

In [ ]:
buggy_elements_df['Number of times modified'] = " "

In [ ]:
from datetime import datetime
from datetime import timedelta

length = buggy_elements_df.shape[0]

for i in range(10000):
    print('i')
    print(i)
    
    try:
        commit_hash = (buggy_elements_df.iloc[i]['Hash'])

        counter = 0

        for j in buggy_elements_df.iloc[i]['Commits']:

            if j == commit_hash:

                break

            else:

                counter = counter + 1

        second_counter = 0

        for date in buggy_elements_df.iloc[i]['Dates']:

            if second_counter == counter:

                recent_date = date

                break

            else:

                second_counter = second_counter + 1

        print(recent_date)

        highest_diff = timedelta(microseconds=-1)

        for date in buggy_elements_df.iloc[i]['Dates']:

            if recent_date - date > highest_diff:

                highest_diff = recent_date - date

            else:

                continue

        (buggy_elements_df.loc[i,['Lifetime']]) =  str(highest_diff)
        (buggy_elements_df.loc[i,['Number of Authors']]) =  str(len(np.unique(buggy_elements_df.iloc[i]['Authors'])))
        (buggy_elements_df.loc[i,['Number of times modified']]) =  str(len(buggy_elements_df.iloc[i]['Commits']))
        
    except IndexError:
        
        print("single positional indexer is out-of-bounds")

        
        
            
    print(highest_diff)        
    print(commit_hash)
    print(counter)

In [ ]:
buggy_elements_df.head()

## Escrita dos dados para um CSV 

In [ ]:
buggy_elements_df.to_csv('..\data\RQ2_Data.csv', index=False, encoding="utf8" )
